### Load project

In [ ]:
from pathlib import Path

from mynd.backend import metashape as metashape
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


CACHE: Path = Path("/data/kingston_snv_01/acfr_point_clouds")
INPUT_DIR: Path = Path("/data/kingston_snv_01/acfr_metashape_projects_dev")
OUTPUT_DIR: Path = Path(
    "/data/kingston_snv_01/acfr_metashape_projects_registered"
)


# NOTE: Change to dense project
INPUT_PROJECT: Path = INPUT_DIR / "r234xgje_dense_with_metadata.psz"
DESTINATION_PROJECT: Path = OUTPUT_DIR / "r234xgje_registered_with_metadata.psz"


assert CACHE.exists(), f"directory does not exist: {CACHE}"
assert OUTPUT_DIR.exists(), f"directory does not exist: {OUTPUT_DIR}"
assert INPUT_PROJECT.exists(), f"project does not exist: {INPUT_PROJECT}"


match metashape.load_project(INPUT_PROJECT):
    case Ok(path):
        logger.info(path)
    case Err(error_message):
        logger.info(error_message)
    case _:
        raise NotImplementedError

In [ ]:
from mynd.geometry import PointCloudProcessor

from mynd.registration import (
    RegistrationPipeline,
    apply_registration_pipeline,
    build_registration_pipeline,
)

from mynd.io import read_config
from mynd.utils.log import logger


CONFIG_FILE: Path = Path(
    "/home/martin/dev/mynd/config/registration_advanced.toml"
)

config: dict = read_config(CONFIG_FILE).unwrap()
registration_config: dict = config.get("registration")


logger.info(registration_config.get("aligner"))

for module in registration_config.get("refiner"):
    logger.info(module)

pipeline: RegistrationPipeline = build_registration_pipeline(
    config.get("registration")
)

In [ ]:
from dataclasses import dataclass

from mynd.collections import GroupID
from mynd.geometry import PointCloud, PointCloudLoader
from mynd.registration import RegistrationResult
from mynd.registration import log_registration_result
from mynd.visualization import visualize_registration


def callback_registration(
    source: PointCloud, target: PointCloud, result: RegistrationResult
) -> None:
    """Callback for registration."""
    log_registration_result(target=target, source=source, result=result)


@dataclass(frozen=True)
class PointCloudHandle:
    """Class representing a point cloud handle."""

    identifier: GroupID
    loader: PointCloudLoader


@dataclass
class ChunkRegistration:
    """Class representing a chunk registration."""

    target: PointCloudHandle
    source: PointCloudHandle
    result: RegistrationResult


# Retrieve dense point clouds
retrieval_result: Result = metashape.dense_services.retrieve_dense_point_clouds(
    cache=CACHE, overwrite=False
)


if retrieval_result.is_err():
    logger.error(retrieval_result.err())

point_cloud_loaders: dict[GroupID, PointCloudLoader] = retrieval_result.ok()

handles: list[PointCloudHandle] = [
    PointCloudHandle(id, loader) for id, loader in point_cloud_loaders.items()
]


target: PointCloudHandle = handles[0]


logger.info("Running registration pipeline")
chunk_registrations: list[ChunkRegistration] = list()
for source in handles[1:]:
    logger.info("")
    logger.info(f" - Target: {target.identifier}")
    logger.info(f" - Source: {source.identifier}")
    logger.info("")

    target_cloud: PointCloud = target.loader().unwrap()
    source_cloud: PointCloud = source.loader().unwrap()

    result: RegistrationResult = apply_registration_pipeline(
        pipeline,
        target=target_cloud,
        source=source_cloud,
        callback=callback_registration,
    )

    chunk_registrations.append(
        ChunkRegistration(target=target, source=source, result=result)
    )

### TODO
[] Add container for registration results
- Write registration results to file
- Update chunk transforms in Metashape

In [ ]:
for registration in chunk_registrations:
    visualize_registration(
        target=registration.target.loader().unwrap(),
        source=registration.source.loader().unwrap(),
        transformation=registration.result.transformation,
    )